In [1]:
import pandas as pd
!pip install openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [openpyxl]1/2 [openpyxl]


In [6]:
df = pd.read_excel("/home/onyxia/work/Statapp_Valuco/feuilles_separees/ValueCo V2_ESG2024.xlsx")

# S'assurer que IVA_RATING_DATE est en datetime
df['IVA_RATING_DATE'] = pd.to_datetime(df['IVA_RATING_DATE'])

# Extraire l'année et le trimestre
df['ANNEE'] = df['IVA_RATING_DATE'].dt.year
df['TRIMESTRE'] = df['IVA_RATING_DATE'].dt.quarter

# Trier pour que la première date du trimestre soit en haut
df = df.sort_values(by=['ISSUER_ISIN', 'ANNEE', 'TRIMESTRE', 'IVA_RATING_DATE'])

# Garder la première ligne de chaque groupe (ISIN + année + trimestre)
df_filtered = df.groupby(['ISSUER_ISIN', 'ANNEE', 'TRIMESTRE'], as_index=False).first()

# Supprimer la colonne IVA_RATING_DATE
df_filtered = df_filtered.drop(columns=['IVA_RATING_DATE'])

# Sauvegarder en CSV
df_filtered.to_csv("msci_trimestre.csv", index=False)

In [3]:
df = pd.read_csv("/home/onyxia/work/Statapp_Valuco/ValueCo data extract - ENSAE research project.csv", parse_dates=["period"])

# Extraire l'année et le trimestre
df["annee"] = df["period"].dt.year
df["trimestre"] = df["period"].dt.quarter

# Trier les données pour que la première date soit en haut de chaque groupe
df = df.sort_values(by=["name", "score_kind", "annee", "trimestre", "period"])

# Garder la première ligne par groupe (name, score_kind, annee, trimestre)
df_filtered = df.groupby(["name", "score_kind", "annee", "trimestre"]).first().reset_index()

# Supprimer la colonne period si elle n'est plus utile
df_filtered = df_filtered.drop(columns=["period"])

# Enregistrer le résultat
df_filtered.to_csv("Valueco_trimestre.csv", index=False)

In [12]:
df = pd.read_csv("/home/onyxia/work/Statapp_Valuco/feuilles_separees/Valueco_trimestre.csv")

# Vérifier que les colonnes année et trimestre sont bien des entiers
df["annee"] = df["annee"].astype(int)
df["trimestre"] = df["trimestre"].astype(int)

# Créer une colonne de tri temporel
df["ordre"] = df["annee"] * 10 + df["trimestre"]

# Trier les données pour le calcul du delta
df = df.sort_values(by=["name", "score_kind", "ordre"])

# Calculer la différence du score avec le trimestre précédent
df["delta"] = df.groupby(["name", "score_kind"])["consensus"].diff()

# Supprimer la colonne temporaire de tri
df = df.drop(columns=["ordre"])

# Sauvegarder le nouveau fichier CSV
df.to_csv("Valueco_avec_delta.csv", index=False)

In [13]:
df = pd.read_csv("/home/onyxia/work/Statapp_Valuco/feuilles_separees/msci_trimestre.csv")

# S'assurer que les colonnes ANNEE et TRIMESTRE sont bien de type entier
df["ANNEE"] = df["ANNEE"].astype(int)
df["TRIMESTRE"] = df["TRIMESTRE"].astype(int)

# Créer une colonne de tri chronologique
df["ordre"] = df["ANNEE"] * 10 + df["TRIMESTRE"]

# Trier correctement
df = df.sort_values(by=["ISSUER_ISIN", "ordre"])

# Liste des scores pour lesquels on veut un delta
scores = [
    "WEIGHTED_AVERAGE_SCORE",
    "ENVIRONMENTAL_PILLAR_SCORE",
    "SOCIAL_PILLAR_SCORE",
    "GOVERNANCE_PILLAR_SCORE"
]

# Calculer les deltas
for score in scores:
    df[f"delta_{score}"] = df.groupby("ISSUER_ISIN")[score].diff()

# Supprimer la colonne temporaire
df = df.drop(columns=["ordre"])

# Sauvegarder dans un nouveau fichier
df.to_csv("MSCI_avec_deltas.csv", index=False)